In [27]:
data_file = "test_result_2016.txt"
data_input = open(data_file, 'r')

for lines in range(3):
    line = data_input.readline()
    print(line)

test_id|vehicle_id|test_date|test_class_id|test_type|test_result|test_mileage|postcode_area|make|model|colour|fuel_type|cylinder_capacity|first_use_date

1645480751|1374211238|2016-01-01|4|NT|P|117033|SM|VOLKSWAGEN|POLO|BLACK|PE|1600|2000-06-23

1393462389|1153769898|2016-01-01|4|NT|P|99292|NE|VOLKSWAGEN|PASSAT|BLUE|DI|1968|2006-11-30



In [30]:
import pandas as pd

# Raw data for 2016 (3GB)
data_file = "test_result_2016.txt"

# Initialize counts
sample_count = 0
testing_validate_count = 0

# Output files
training_file = "MOT_result_2016_training.csv"
validation_file = "MOT_result_2016_validation.csv"
testing_file = "MOT_result_2016_testing.csv"

# Use write mode and headers for 1st dataframe only
mode = 'w'
header = True

# Process the file in chunks
chunksize = 10**4

for chunk in pd.read_csv(data_file, sep='|', chunksize=chunksize, error_bad_lines=False):
    if mode == 'w': # 1st dataframe sampled, set random state
        chunk = chunk.query('test_class_id == 4').sample(frac=0.01, random_state = 21)
    else:
        chunk = chunk.query('test_class_id == 4').sample(frac=0.01)
    # Set aside 10% for Testing, 10% for Validation, 80% for training    
    chunk_len = len(chunk)
    testing_validate_len = chunk_len // 10
    chunk[:testing_validate_len].to_csv(testing_file, index=False, mode=mode, header=header)
    chunk[(testing_validate_len + 1):(testing_validate_len * 2)].to_csv(validation_file, index=False, mode=mode, header=header)
    chunk[(testing_validate_len * 2 + 1):].to_csv(training_file, index=False, mode=mode, header=header)
    # Set mode for writing CSV file to append for subsequent samples
    mode = 'a'
    # Update counts
    sample_count += chunk_len
    testing_validate_count += testing_validate_len

# Subtracting testing and validation counts to get training counts
training_count = sample_count - 2 * testing_validate_count
print('The number of samples taken was ' + "{:,}".format(sample_count))
print('This was split as follows:')
print('  Training - ' + "{:,}".format(training_count))
print('  Validation - ' + "{:,}".format(testing_validate_count))
print('  Testing - ' + "{:,}".format(testing_validate_count))

b'Skipping line 35021189: expected 14 fields, saw 16\n'
b'Skipping line 37070120: expected 14 fields, saw 16\n'


The number of samples taken was 356,383
This was split as follows:
  Training - 288,675
  Validation - 33,854
  Testing - 33,854


In [65]:
import re
postcode_file = 'National_Statistics_Postcode_Lookup_UK.csv'
#postcode_file = 'postcode sample.csv'
postcode_area_file = 'Postcode Area.csv'

postcode_df = pd.read_csv(postcode_file)
postcode_df['Postcode Area'] = (postcode_df['Postcode 3'].str.extract('([A-Z]+)', expand=True))
postcode_df = postcode_df.groupby(['Postcode Area']).first()[['Local Authority Name', 'Country Name', 'Region Name']]


postcode_df.loc[postcode_df['Country Name'] == 'England','Country_Region'] = postcode_df.loc[postcode_df['Country Name'] == 'England','Region Name']
postcode_df.loc[postcode_df['Country Name'] != 'England','Country_Region'] = postcode_df.loc[postcode_df['Country Name'] != 'England','Country Name']
postcode_df.to_csv(postcode_area_file)
postcode_df


,Local Authority Name,Country Name,Region Name,Country_Region
Postcode Area,,,,
AB,Aberdeen City,Scotland,NaN,Scotland
AL,St Albans,England,East of England,East of England
B,Birmingham,England,West Midlands,West Midlands
BA,Bath and North East Somerset,England,South West,South West
BB,Hyndburn,England,North West,North West
BD,Bradford,England,Yorkshire and The Humber,Yorkshire and The Humber
BH,Poole,England,South West,South West
BL,Bolton,England,North West,North West
BN,Lewes,England,South East,South East
